# AITH DL Competition - Multi-Model Toxicity Classification Pipeline

Продвинутый пайплайн для классификации токсичных комментариев с использованием 4 моделей.

**Модели:**
1. `textdetox/xlmr-large-toxicity-classifier-v2` — топ по устойчивости к "грязному" тексту
2. `s-nlp/russian_toxicity_classifier` — сильный RU-интернет бейзлайн
3. `IlyaGusev/rubertconv_toxic_clf` — хорош для диалогов/переписок
4. `cointegrated/rubert-tiny-toxicity` — быстрый и лёгкий

In [1]:
%pip install -q transformers torch pandas accelerate sentencepiece optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 15.3 MB/s eta 0:00:00


In [2]:
import os
import re
import gc
import pandas as pd
import numpy as np
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from tqdm.auto import tqdm
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

os.environ["TOKENIZERS_PARALLELISM"] = "false"

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("CUDA optimizations enabled: cuDNN benchmark, TF32")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
GPU Memory: 42.5 GB
CUDA optimizations enabled: cuDNN benchmark, TF32


## Конфигурация моделей

In [3]:
MODELS_CONFIG = {
    "xlmr_large_v2": {
        "name": "textdetox/xlmr-large-toxicity-classifier-v2",
        "batch_size": 256,
        "max_length": 256,
        "toxic_label": "toxic",
        "description": "XLM-R large v2 - топ по устойчивости к грязному тексту"
    },
    "snlp_russian": {
        "name": "s-nlp/russian_toxicity_classifier",
        "batch_size": 512,
        "max_length": 256,
        "toxic_label": "toxic",
        "description": "s-nlp Russian toxicity - сильный RU бейзлайн"
    },
    "rubertconv": {
        "name": "IlyaGusev/rubertconv_toxic_clf",
        "batch_size": 512,
        "max_length": 256,
        "toxic_label": "toxic",
        "description": "RuBERT conversational - хорош для диалогов"
    },
    "rubert_tiny": {
        "name": "cointegrated/rubert-tiny-toxicity",
        "batch_size": 1024,
        "max_length": 256,
        "toxic_label": "toxic",
        "description": "RuBERT tiny - быстрый и лёгкий"
    }
}

print("Модели для обработки (оптимизировано для A100):")
for key, config in MODELS_CONFIG.items():
    print(f"  • {key}: batch={config['batch_size']}, max_len={config['max_length']}")

Модели для обработки (оптимизировано для A100):
  • xlmr_large_v2: batch=256, max_len=256
  • snlp_russian: batch=512, max_len=256
  • rubertconv: batch=512, max_len=256
  • rubert_tiny: batch=1024, max_len=256


## Preprocessing - нормализация текста

In [4]:
TRANSLIT_MAP = {
    'a': 'а', 'b': 'б', 'c': 'с', 'e': 'е', 'k': 'к', 'm': 'м',
    'n': 'н', 'o': 'о', 'p': 'р', 't': 'т', 'x': 'х', 'y': 'у',
    'h': 'н', 'u': 'у', 'i': 'и'
}

def normalize_text(text: str,
                   do_lower: bool = True,
                   remove_repeated: bool = True,
                   remove_masks: bool = True,
                   do_translit: bool = False) -> str:
    """
    Нормализация текста для улучшения качества классификации.

    Args:
        text: Исходный текст
        do_lower: Привести к нижнему регистру
        remove_repeated: Убрать повторяющиеся символы (ууужас -> ужас)
        remove_masks: Убрать маскирующие символы (*, #, @)
        do_translit: Конвертировать латиницу в кириллицу
    """
    if not isinstance(text, str):
        return ""

    result = text

    if do_lower:
        result = result.lower()

    if remove_masks:
        result = re.sub(r'(?<=[а-яёa-z])[-*#@$%!_.]+(?=[а-яёa-z])', '', result, flags=re.IGNORECASE)
        result = re.sub(r'\s+', ' ', result)

    if remove_repeated:
        result = re.sub(r'(.)\1{2,}', r'\1\1', result)

    if do_translit:
        chars = list(result)
        for i, char in enumerate(chars):
            if char in TRANSLIT_MAP:
                chars[i] = TRANSLIT_MAP[char]
        result = ''.join(chars)

    return result.strip()


def preprocess_texts(texts: List[str], normalize: bool = True) -> List[str]:
    """Batch preprocessing текстов."""
    if normalize:
        return [normalize_text(t, do_lower=True, remove_repeated=True,
                               remove_masks=True, do_translit=False) for t in texts]
    return texts


test_texts = [
    "Ты п*здец тупой!",
    "Какой же ты дееееебил",
    "Н_о_р_м_а_л_ь_н_о",
    "КАПС ЛОЧНЫЙ ТЕКСТ",
    "blyat pizdec"
]

print("Примеры нормализации:")
for t in test_texts:
    print(f"  '{t}' -> '{normalize_text(t)}'")

Примеры нормализации:
  'Ты п*здец тупой!' -> 'ты пздец тупой!'
  'Какой же ты дееееебил' -> 'какой же ты деебил'
  'Н_о_р_м_а_л_ь_н_о' -> 'нормально'
  'КАПС ЛОЧНЫЙ ТЕКСТ' -> 'капс лочный текст'
  'blyat pizdec' -> 'blyat pizdec'


## Загрузка данных

In [5]:
DATA_PATH = "aith-dl-competition-text-data-2025/"
if not os.path.exists(DATA_PATH):
    DATA_PATH = "./"

test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
test_df['text'] = test_df['text'].fillna('')

print("Применяем preprocessing...")
test_df['text_normalized'] = preprocess_texts(test_df['text'].tolist(), normalize=True)

print(f"\nTest size: {len(test_df)}")
print(f"\nПримеры данных:")
for i in range(min(5, len(test_df))):
    orig = test_df.iloc[i]['text'][:50]
    norm = test_df.iloc[i]['text_normalized'][:50]
    print(f"  Original: {orig}...")
    print(f"  Normalized: {norm}...")
    print()

Применяем preprocessing...

Test size: 74487

Примеры данных:
  Original: господь услышал слова людей, наши молитвы дошли !!...
  Normalized: господь услышал слова людей, наши молитвы дошли !!...

  Original: их тоже надо выстрелами в головы пристрелить.без в...
  Normalized: их тоже надо выстрелами в головы пристрелитьбез вс...

  Original: не ужели не понятно, он так завтракает, обедает и ...
  Normalized: не ужели не понятно, он так завтракает, обедает и ...

  Original: я открыла, прочитала...
  Normalized: я открыла, прочитала...

  Original: нет ты давай по взрослому с помадой :p :d...
  Normalized: нет ты давай по взрослому с помадой :p :d...



In [6]:
def get_toxic_score(result: Dict, model_key: str) -> float:
    """
    Извлекает вероятность токсичности из результата модели.
    Разные модели могут иметь разные форматы выхода.
    """
    label = result.get('label', '').lower()
    score = result.get('score', 0.0)

    toxic_labels = ['toxic', 'positive', '1', 'label_1', 'toxic_1']
    non_toxic_labels = ['neutral', 'negative', '0', 'label_0', 'non-toxic', 'non_toxic']

    if any(tl in label for tl in toxic_labels):
        return score
    elif any(ntl in label for ntl in non_toxic_labels):
        return 1 - score
    else:
        print(f"Warning: Unknown label format '{label}' for {model_key}")
        return score


def predict_with_model_fast(model_key: str,
                            texts: List[str]) -> Tuple[List[int], List[float]]:
    """
    БЫСТРОЕ предсказание с использованием указанной модели.

    Оптимизации:
    - BetterTransformer (Flash Attention)
    - torch.inference_mode()
    - torch.compile() для PyTorch 2.0+
    - Оптимизированный батчинг

    Returns:
        Tuple[List[int], List[float]]: (binary predictions, toxic probabilities)
    """
    import time
    start_time = time.time()

    config = MODELS_CONFIG[model_key]
    model_name = config['name']
    batch_size = config['batch_size']
    max_length = config['max_length']

    print(f"\n{'='*60}")
    print(f"🚀 FAST MODE: {model_key}")
    print(f"  Model: {model_name}")
    print(f"  Batch size: {batch_size}")
    print(f"  Max length: {max_length}")
    print(f"{'='*60}")

    print("Загружаем модель...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="cuda:0"
    )

    try:
        model = model.to_bettertransformer()
        print("✓ BetterTransformer enabled")
    except Exception as e:
        print(f"✗ BetterTransformer not available: {e}")

    if hasattr(torch, 'compile'):
        try:
            model = torch.compile(model, mode="reduce-overhead")
            print("✓ torch.compile() enabled")
        except Exception as e:
            print(f"✗ torch.compile() not available: {e}")

    model.eval()
    print(f"GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

    with torch.inference_mode():
        test_inputs = tokenizer(["тест"], return_tensors="pt",
                                truncation=True, max_length=max_length, padding=True)
        test_inputs = {k: v.cuda() for k, v in test_inputs.items()}
        test_outputs = model(**test_inputs)
        test_probs = torch.softmax(test_outputs.logits, dim=-1)
        print(f"Output shape: {test_probs.shape}, Labels: {model.config.id2label}")

    id2label = model.config.id2label
    toxic_idx = None
    for idx, label in id2label.items():
        if 'toxic' in label.lower() or label in ['1', 'LABEL_1', 'positive']:
            toxic_idx = idx
            break
    if toxic_idx is None:
        toxic_idx = 1
    print(f"Toxic class index: {toxic_idx}")

    all_scores = []

    with torch.inference_mode():
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Predicting ({model_key})"):
            batch_texts = texts[i:i+batch_size]

            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                truncation=True,
                max_length=max_length,
                padding=True
            )
            inputs = {k: v.cuda() for k, v in inputs.items()}

            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=-1)

            toxic_probs = probs[:, toxic_idx].cpu().numpy()
            all_scores.extend(toxic_probs.tolist())

    all_preds = [1 if s >= 0.5 else 0 for s in all_scores]

    elapsed = time.time() - start_time
    samples_per_sec = len(texts) / elapsed

    print(f"\n✅ Готово за {elapsed:.1f} сек ({samples_per_sec:.0f} samples/sec)")
    print(f"Распределение: 0={sum(p==0 for p in all_preds)}, 1={sum(p==1 for p in all_preds)}")
    print(f"Средняя вероятность токсичности: {np.mean(all_scores):.4f}")

    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()
    print(f"GPU memory после очистки: {torch.cuda.memory_allocated()/1e9:.2f} GB")

    return all_preds, all_scores


predict_with_model = predict_with_model_fast


def save_submission(test_df: pd.DataFrame,
                    predictions: List[int],
                    scores: List[float],
                    model_key: str,
                    use_probabilities: bool = False) -> str:
    """Сохраняет submission файл."""
    submission = pd.DataFrame({
        'id': test_df['id'],
        'score': scores if use_probabilities else predictions
    })

    filename = f'submission_{model_key}.csv'
    submission.to_csv(filename, index=False)

    print(f"\nSubmission сохранён: {filename}")
    print(submission.head())
    if not use_probabilities:
        print(f"Distribution: {submission['score'].value_counts().to_dict()}")
    else:
        print(f"Score stats: min={submission['score'].min():.4f}, max={submission['score'].max():.4f}, mean={submission['score'].mean():.4f}")

    return filename

## 1. Модель: textdetox/xlmr-large-toxicity-classifier-v2

In [7]:
texts_to_predict = test_df['text_normalized'].tolist()

preds_xlmr, scores_xlmr = predict_with_model(
    model_key="xlmr_large_v2",
    texts=texts_to_predict
)

save_submission(test_df, preds_xlmr, scores_xlmr, "xlmr_large_v2")


🚀 FAST MODE: xlmr_large_v2
  Model: textdetox/xlmr-large-toxicity-classifier-v2
  Batch size: 256
  Max length: 256
Загружаем модель...


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

...

✗ BetterTransformer not available: No module named 'optimum.bettertransformer'
✓ torch.compile() enabled
GPU memory: 1.12 GB
Output shape: torch.Size([1, 2]), Labels: {0: 'LABEL_0', 1: 'LABEL_1'}
Toxic class index: 1


Predicting (xlmr_large_v2):   0%|          | 0/291 [00:00<?, ?it/s]

W0124 19:47:06.476000 2158 torch/fx/experimental/symbolic_shapes.py:6833] [0/1] _maybe_guard_rel() was called on non-relation expression Eq(s43, 1) | Eq(s72, s43)
W0124 19:47:06.481000 2158 torch/fx/experimental/symbolic_shapes.py:6833] [0/1] _maybe_guard_rel() was called on non-relation expression Eq(s53, 1) | Eq(s70, s53)
CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.



✅ Готово за 171.5 сек (434 samples/sec)
Распределение: 0=56682, 1=17805
Средняя вероятность токсичности: 0.2407
GPU memory после очистки: 0.00 GB

Submission сохранён: submission_xlmr_large_v2.csv
       id  score
0  152591      0
1  108771      1
2  198853      0
3  194736      0
4   88110      0
Distribution: {0: 56682, 1: 17805}


'submission_xlmr_large_v2.csv'

## 2. Модель: s-nlp/russian_toxicity_classifier


In [8]:
preds_snlp, scores_snlp = predict_with_model(
    model_key="snlp_russian",
    texts=texts_to_predict
)

save_submission(test_df, preds_snlp, scores_snlp, "snlp_russian")


🚀 FAST MODE: snlp_russian
  Model: s-nlp/russian_toxicity_classifier
  Batch size: 512
  Max length: 256
Загружаем модель...


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

✗ BetterTransformer not available: No module named 'optimum.bettertransformer'
✓ torch.compile() enabled
GPU memory: 0.36 GB
Output shape: torch.Size([1, 2]), Labels: {0: 'neutral', 1: 'toxic'}
Toxic class index: 1


Predicting (snlp_russian):   0%|          | 0/146 [00:00<?, ?it/s]

W0124 19:49:33.055000 2158 torch/fx/experimental/symbolic_shapes.py:6833] [1/1] _maybe_guard_rel() was called on non-relation expression Eq(s18, s43) | Eq(s43, 1)
W0124 19:49:33.059000 2158 torch/fx/experimental/symbolic_shapes.py:6833] [1/1] _maybe_guard_rel() was called on non-relation expression Eq(s41, s53) | Eq(s53, 1)
CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.



✅ Готово за 77.5 сек (961 samples/sec)
Распределение: 0=61167, 1=13320
Средняя вероятность токсичности: 0.1789
GPU memory после очистки: 0.00 GB

Submission сохранён: submission_snlp_russian.csv
       id  score
0  152591      0
1  108771      1
2  198853      0
3  194736      0
4   88110      0
Distribution: {0: 61167, 1: 13320}


'submission_snlp_russian.csv'

## 3. Модель: IlyaGusev/rubertconv_toxic_clf


In [9]:
preds_conv, scores_conv = predict_with_model(
    model_key="rubertconv",
    texts=texts_to_predict
)

save_submission(test_df, preds_conv, scores_conv, "rubertconv")


🚀 FAST MODE: rubertconv
  Model: IlyaGusev/rubertconv_toxic_clf
  Batch size: 512
  Max length: 256
Загружаем модель...


tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

✗ BetterTransformer not available: No module named 'optimum.bettertransformer'
✓ torch.compile() enabled
GPU memory: 0.36 GB
Output shape: torch.Size([1, 2]), Labels: {0: 'neutral', 1: 'toxic'}
Toxic class index: 1


Predicting (rubertconv):   0%|          | 0/146 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]


✅ Готово за 72.0 сек (1034 samples/sec)
Распределение: 0=60378, 1=14109
Средняя вероятность токсичности: 0.1914
GPU memory после очистки: 0.00 GB

Submission сохранён: submission_rubertconv.csv
       id  score
0  152591      0
1  108771      1
2  198853      0
3  194736      0
4   88110      0
Distribution: {0: 60378, 1: 14109}


'submission_rubertconv.csv'

## 4. Модель: cointegrated/rubert-tiny-toxicity


In [10]:
preds_tiny, scores_tiny = predict_with_model(
    model_key="rubert_tiny",
    texts=texts_to_predict
)

save_submission(test_df, preds_tiny, scores_tiny, "rubert_tiny")


🚀 FAST MODE: rubert_tiny
  Model: cointegrated/rubert-tiny-toxicity
  Batch size: 1024
  Max length: 256
Загружаем модель...


tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

✗ BetterTransformer not available: No module named 'optimum.bettertransformer'
✓ torch.compile() enabled
GPU memory: 0.02 GB
Output shape: torch.Size([1, 5]), Labels: {0: 'non-toxic', 1: 'insult', 2: 'obscenity', 3: 'threat', 4: 'dangerous'}
Toxic class index: 0


Predicting (rubert_tiny):   0%|          | 0/73 [00:00<?, ?it/s]


✅ Готово за 91.1 сек (818 samples/sec)
Распределение: 0=14904, 1=59583
Средняя вероятность токсичности: 0.7958
GPU memory после очистки: 0.00 GB

Submission сохранён: submission_rubert_tiny.csv
       id  score
0  152591      1
1  108771      0
2  198853      1
3  194736      1
4   88110      0
Distribution: {1: 59583, 0: 14904}


'submission_rubert_tiny.csv'

## Ансамбль моделей

Создаём несколько вариантов ансамблей:
1. **Усреднение вероятностей** — среднее по всем моделям
2. **Взвешенное усреднение** — больший вес робастным моделям
3. **Max voting** — если хотя бы одна модель уверена в токсичности
4. **Majority voting** — большинство голосов

In [11]:
results_df = pd.DataFrame({
    'id': test_df['id'],
    'text': test_df['text'],
    'text_normalized': test_df['text_normalized'],
    'score_xlmr': scores_xlmr,
    'score_snlp': scores_snlp,
    'score_conv': scores_conv,
    'score_tiny': scores_tiny,
    'pred_xlmr': preds_xlmr,
    'pred_snlp': preds_snlp,
    'pred_conv': preds_conv,
    'pred_tiny': preds_tiny,
})

print("Результаты всех моделей:")
print(results_df.head(10))

Результаты всех моделей:
       id                                               text  \
0  152591  господь услышал слова людей, наши молитвы дошл...   
1  108771  их тоже надо выстрелами в головы пристрелить.б...   
2  198853  не ужели не понятно, он так завтракает, обедае...   
3  194736                               я открыла, прочитала   
4   88110          нет ты давай по взрослому с помадой :p :d   
5  154244                  из двух яиц. 2-фото - с куркумой.   
6  200659  восхищена поступком ! какой молодец ! берите п...   
7  130248    дувушки в болталку, или сейчас вас занесу в бан   
8  167595  какая лапуличка сидит я тоже хочу побыть на ег...   
9  233620  мы так и взяли. третий месяц растет теленок, н...   

                                     text_normalized  score_xlmr  score_snlp  \
0  господь услышал слова людей, наши молитвы дошл...    0.000606    0.000298   
1  их тоже надо выстрелами в головы пристрелитьбе...    0.997559    0.992676   
2  не ужели не понятно, он так

In [12]:
results_df['score_avg'] = (
    results_df['score_xlmr'] +
    results_df['score_snlp'] +
    results_df['score_conv'] +
    results_df['score_tiny']
) / 4
results_df['pred_avg'] = (results_df['score_avg'] >= 0.5).astype(int)

weights = {
    'xlmr': 0.35,
    'snlp': 0.30,
    'conv': 0.20,
    'tiny': 0.15
}
results_df['score_weighted'] = (
    results_df['score_xlmr'] * weights['xlmr'] +
    results_df['score_snlp'] * weights['snlp'] +
    results_df['score_conv'] * weights['conv'] +
    results_df['score_tiny'] * weights['tiny']
)
results_df['pred_weighted'] = (results_df['score_weighted'] >= 0.5).astype(int)

CONFIDENCE_THRESHOLD = 0.7
results_df['score_max'] = results_df[['score_xlmr', 'score_snlp', 'score_conv', 'score_tiny']].max(axis=1)
results_df['pred_max'] = (results_df['score_max'] >= CONFIDENCE_THRESHOLD).astype(int)

results_df['votes'] = (
    results_df['pred_xlmr'] +
    results_df['pred_snlp'] +
    results_df['pred_conv'] +
    results_df['pred_tiny']
)
results_df['pred_majority'] = (results_df['votes'] >= 2).astype(int)  # >= 2 из 4 моделей

print("Статистика ансамблей:")
print(f"  Average: toxic={results_df['pred_avg'].sum()}, non-toxic={(results_df['pred_avg']==0).sum()}")
print(f"  Weighted: toxic={results_df['pred_weighted'].sum()}, non-toxic={(results_df['pred_weighted']==0).sum()}")
print(f"  Max (conf>{CONFIDENCE_THRESHOLD}): toxic={results_df['pred_max'].sum()}, non-toxic={(results_df['pred_max']==0).sum()}")
print(f"  Majority: toxic={results_df['pred_majority'].sum()}, non-toxic={(results_df['pred_majority']==0).sum()}")

Статистика ансамблей:
  Average: toxic=14573, non-toxic=59914
  Weighted: toxic=14757, non-toxic=59730
  Max (conf>0.7): toxic=73457, non-toxic=1030
  Majority: toxic=17816, non-toxic=56671


In [13]:
submission_avg = pd.DataFrame({
    'id': results_df['id'],
    'score': results_df['pred_avg']
})
submission_avg.to_csv('submission_ensemble_avg.csv', index=False)
print("Saved: submission_ensemble_avg.csv")
print(f"  Distribution: {submission_avg['score'].value_counts().to_dict()}")

submission_weighted = pd.DataFrame({
    'id': results_df['id'],
    'score': results_df['pred_weighted']
})
submission_weighted.to_csv('submission_ensemble_weighted.csv', index=False)
print("\nSaved: submission_ensemble_weighted.csv")
print(f"  Distribution: {submission_weighted['score'].value_counts().to_dict()}")

submission_max = pd.DataFrame({
    'id': results_df['id'],
    'score': results_df['pred_max']
})
submission_max.to_csv('submission_ensemble_max.csv', index=False)
print("\nSaved: submission_ensemble_max.csv")
print(f"  Distribution: {submission_max['score'].value_counts().to_dict()}")

submission_majority = pd.DataFrame({
    'id': results_df['id'],
    'score': results_df['pred_majority']
})
submission_majority.to_csv('submission_ensemble_majority.csv', index=False)
print("\nSaved: submission_ensemble_majority.csv")
print(f"  Distribution: {submission_majority['score'].value_counts().to_dict()}")

Saved: submission_ensemble_avg.csv
  Distribution: {0: 59914, 1: 14573}

Saved: submission_ensemble_weighted.csv
  Distribution: {0: 59730, 1: 14757}

Saved: submission_ensemble_max.csv
  Distribution: {1: 73457, 0: 1030}

Saved: submission_ensemble_majority.csv
  Distribution: {0: 56671, 1: 17816}


## Анализ расхождений между моделями

In [14]:
results_df['disagreement'] = results_df[['pred_xlmr', 'pred_snlp', 'pred_conv', 'pred_tiny']].std(axis=1)

disagreements = results_df[results_df['disagreement'] > 0.4].sort_values('disagreement', ascending=False)

print(f"Случаев с сильным расхождением: {len(disagreements)} из {len(results_df)}")
print("\nПримеры расхождений:")
for i, row in disagreements.head(10).iterrows():
    print(f"\nText: {row['text'][:80]}...")
    print(f"  XLM-R: {row['score_xlmr']:.3f} ({row['pred_xlmr']})")
    print(f"  s-nlp: {row['score_snlp']:.3f} ({row['pred_snlp']})")
    print(f"  Conv:  {row['score_conv']:.3f} ({row['pred_conv']})")
    print(f"  Tiny:  {row['score_tiny']:.3f} ({row['pred_tiny']})")

Случаев с сильным расхождением: 73704 из 74487

Примеры расхождений:

Text: гнать этих депутатов и чиновников в шею, позор таким, нет бы людям помочь а они ...
  XLM-R: 0.987 (1)
  s-nlp: 0.034 (0)
  Conv:  0.004 (0)
  Tiny:  0.987 (1)

Text: охх какию красота я хочу селе ден сосать...
  XLM-R: 0.004 (0)
  s-nlp: 0.989 (1)
  Conv:  0.940 (1)
  Tiny:  0.001 (0)

Text: с вами я на ты не переходил, и терпеть нравоучений от полуграмотного быдла не ум...
  XLM-R: 1.000 (1)
  s-nlp: 0.037 (0)
  Conv:  0.550 (1)
  Tiny:  0.141 (0)

Text: бред. на корню зарубим я с хабаровска...
  XLM-R: 0.979 (1)
  s-nlp: 0.074 (0)
  Conv:  0.110 (0)
  Tiny:  0.998 (1)

Text: только не плакайте, когда вас будут стрелять....
  XLM-R: 0.004 (0)
  s-nlp: 0.953 (1)
  Conv:  0.997 (1)
  Tiny:  0.000 (0)

Text: а это не ваша власть это оккупанты , своя власть так не делает на китай посмотри...
  XLM-R: 0.757 (1)
  s-nlp: 0.007 (0)
  Conv:  0.011 (0)
  Tiny:  0.999 (1)

Text: вот кто настоящий ряженный , клоун...
  

In [15]:
print("Корреляция вероятностей между моделями:")
correlation_cols = ['score_xlmr', 'score_snlp', 'score_conv', 'score_tiny']
corr_matrix = results_df[correlation_cols].corr()
print(corr_matrix.round(3))

Корреляция вероятностей между моделями:
            score_xlmr  score_snlp  score_conv  score_tiny
score_xlmr       1.000       0.821       0.844      -0.839
score_snlp       0.821       1.000       0.953      -0.931
score_conv       0.844       0.953       1.000      -0.931
score_tiny      -0.839      -0.931      -0.931       1.000


## Итоговая сводка

In [16]:
submissions = [
    ('submission_xlmr_large_v2.csv', 'XLM-R large v2 - робастная модель'),
    ('submission_snlp_russian.csv', 's-nlp Russian toxicity - RU бейзлайн'),
    ('submission_rubertconv.csv', 'RuBERT conversational - для диалогов'),
    ('submission_rubert_tiny.csv', 'RuBERT tiny - быстрая модель'),
    ('submission_ensemble_avg.csv', 'Ансамбль - простое усреднение'),
    ('submission_ensemble_weighted.csv', 'Ансамбль - взвешенное усреднение'),
    ('submission_ensemble_max.csv', f'Ансамбль - max voting (conf>{CONFIDENCE_THRESHOLD})'),
    ('submission_ensemble_majority.csv', 'Ансамбль - majority voting'),
]

print("="*70)
print("ИТОГОВАЯ СВОДКА - СОЗДАННЫЕ SUBMISSION ФАЙЛЫ")
print("="*70)

for filename, description in submissions:
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        toxic_count = (df['score'] == 1).sum() if df['score'].max() <= 1 else (df['score'] >= 0.5).sum()
        print(f"\n{filename}")
        print(f"  {description}")
        print(f"  Toxic: {toxic_count}, Non-toxic: {len(df) - toxic_count}")

print("\n" + "="*70)
print("Рекомендация: начните с submission_ensemble_weighted.csv или submission_xlmr_large_v2.csv")
print("="*70)

ИТОГОВАЯ СВОДКА - СОЗДАННЫЕ SUBMISSION ФАЙЛЫ

submission_xlmr_large_v2.csv
  XLM-R large v2 - робастная модель
  Toxic: 17805, Non-toxic: 56682

submission_snlp_russian.csv
  s-nlp Russian toxicity - RU бейзлайн
  Toxic: 13320, Non-toxic: 61167

submission_rubertconv.csv
  RuBERT conversational - для диалогов
  Toxic: 14109, Non-toxic: 60378

submission_rubert_tiny.csv
  RuBERT tiny - быстрая модель
  Toxic: 59583, Non-toxic: 14904

submission_ensemble_avg.csv
  Ансамбль - простое усреднение
  Toxic: 14573, Non-toxic: 59914

submission_ensemble_weighted.csv
  Ансамбль - взвешенное усреднение
  Toxic: 14757, Non-toxic: 59730

submission_ensemble_max.csv
  Ансамбль - max voting (conf>0.7)
  Toxic: 73457, Non-toxic: 1030

submission_ensemble_majority.csv
  Ансамбль - majority voting
  Toxic: 17816, Non-toxic: 56671

Рекомендация: начните с submission_ensemble_weighted.csv или submission_xlmr_large_v2.csv


In [17]:
results_df.to_csv('all_models_predictions.csv', index=False)
print("Сохранено: all_models_predictions.csv")
print("Этот файл содержит предсказания всех моделей для каждого текста.")

Сохранено: all_models_predictions.csv
Этот файл содержит предсказания всех моделей для каждого текста.


#Что еще пробовал:

1.   Full finetuning, linear probing, lora adapter для моделей cointegrated/rubert-tiny-toxicity, DeepPavlov/rubert-base-cased, ai-forever/ruRoberta-large, submission_xlmr_large_v2
2.   Различные Аугментации данных и предобработки данных

*   исправление искажённых ругательств
*   замена латиницы на кириллицу в мате
*   сжатие повторяющихся букв
*   KeyboardAug
*   RandomCharAug
*   и др


3.   Пытался искать примеры на которых модель чаще всего ошибается (False negative, False positive, Пограничные случаи классификации, Анализ слов в ошибочных текстах и добавление примеров/аугментаций с ними)
4.   Перебирал гиперпараметры, трешхолды по которым происходит разделение по классам
5.   Пробовал обучать простенькие модели с нуля (BidirectionalLstm сходу дал 92% качества)

В итоге лучший результат дала просто классификация через пайплайны с предобученными для классификации моделями 🙂

А еще я не знал что больше 20 сабмишенов в день посылать нельзя🙂, из за этого скор только 0.97842, c файлом submission_rubertconv.csv, возможно какой то ансамбль бы дал большее качество 😔

